In [127]:
import pandas as pd
import folium
import os

# 读取数据
df = pd.read_excel("./储能火灾事故分析.xlsx")

# Delete NA rows
df = df[df['经纬度坐标'].notna()]

# 假设列名是 'Custom location (Lat, Lon)'
df[['Latitude', 'Longitude']] = df['经纬度坐标'].str.split(',', expand=True)

# 转换为浮点数
df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace('\ufeff', '').str.strip(), errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'].str.strip(), errors='coerce')

In [128]:
# 定义分类函数
def classify_battery_type(x):
    if pd.isna(x):
        return '其他'
    
    x = str(x).lower().replace(' ', '')  # 转小写并去空格
    
    if any(keyword in x for keyword in ['磷酸铁锂', 'lfp']):
        return '磷酸铁锂'
    elif any(keyword in x for keyword in ['三元', 'nmc', 'nca']):
        return '三元锂'
    elif any(keyword in x for keyword in ['NaS']):
        return '钠硫'
    elif any(keyword in x for keyword in ['PbA']):
        return '铅酸'
    else:
        return 'unknown'

# 应用分类函数
df['电池类型分类'] = df['电池类型（三元/铁锂）'].apply(classify_battery_type)

In [129]:
# 确保时间列是 datetime 类型
df['时间'] = df['时间'].astype(str)
df['时间'] = pd.to_datetime(df['时间'], format="%Y-%m-%d", errors='coerce')

# 按时间升序排序
df = df.sort_values(by='时间', ascending=True).reset_index(drop=True)

In [131]:
m = folium.Map(location=[20, 0], zoom_start=2)

for _, row in df.iterrows():
    popup_text = f"""
    <b>Event Date:</b> {row['时间']}<br>
    <b>Capacity (MWh):</b> {row['规模 MWh']}<br>
    <b>Battery Modules:</b> {row['电池类型（三元/铁锂）']}<br>
    <b>Root Cause:</b> {row['直接原因']}
    """
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_text,
        icon=folium.Icon(color='red', icon='fire')
    ).add_to(m)

# 输出 HTML 文件到 templates 文件夹（供 Flask 使用）
os.makedirs("templates", exist_ok=True)

In [132]:
m.save("templates/storage_fire_map.html")